In [25]:
#!pip install ultralytics

In [26]:
import ultralytics
ultralytics.__version__

'8.1.43'

In [27]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import *

model=YOLO('yolov8n.pt')

In [28]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [29]:
tracker=Tracker()
counter=0

In [30]:
cap=cv2.VideoCapture('test.mp4')

In [31]:
down={}
up={}

counter_down=[]
counter_up=[]

In [32]:
while True:    
    ret,frame = cap.read()
    if not ret:
        break
    # count += 1
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
    
#     print(results)
     
    a=results[0].boxes.data
    a = a.detach().cpu().numpy()  
    px=pd.DataFrame(a).astype("float")
    
#     print(px)

    list=[]
             
    for index,row in px.iterrows():
        print(row) 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list.append([x1,y1,x2,y2])
#             print(c)

    bbox_id=tracker.update(list)
    #print(bbox_id)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2

        red_line_y=198
        blue_line_y=268   
        offset = 7
        
  

        ''' both lines combined condition . First condition is for red line'''
        ## condition for counting the cars which are entering from red line and exiting from blue line
        if red_line_y < (cy + offset) and red_line_y > (cy - offset):
          down[id]=cy   
        if id in down:
           if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):         
             cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
             cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
#              counter+=1
             counter_down.append(id)  # get a list of the cars and buses which are entering the line red and exiting the line blue

        # condition for cars entering from  blue line
        if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
          up[id]=cy   
        if id in up:
           if red_line_y < (cy + offset) and red_line_y > (cy - offset):         
             cv2.circle(frame,(cx,cy),4,(0,0,255),-1)
             cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
             #counter+=1
             counter_up.append(id)  # get a list of the cars which are entering the line 1 and exiting the line 2 




    
    text_color = (255,255,255)  # white color for text
    red_color = (0, 0, 255)  # (B, G, R)   
    blue_color = (255, 0, 0)  # (B, G, R)
    green_color = (0, 255, 0)  # (B, G, R)  

    cv2.line(frame,(172,198),(774,198),red_color,3)  #  starting cordinates and end of line cordinates
    cv2.putText(frame,('red line'),(172,198),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    
    cv2.line(frame,(8,268),(927,268),blue_color,3)  # seconde line
    cv2.putText(frame,('blue line'),(8,268),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)    


    downwards = (len(counter_down))
    # print("Printing Downwards :- ",downwards)
    cv2.putText(frame,('going down - ')+ str(downwards),(60,40),cv2.FONT_HERSHEY_SIMPLEX, 0.5, green_color, 1, cv2.LINE_AA)    
    
    
    upwards = (len(counter_up))
    # print("Printing Downwards :- ",upwards)
    cv2.putText(frame,('going up - ')+ str(upwards),(60,60),cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)  

    cv2.imshow("frames", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()


0: 320x640 2 persons, 7 cars, 1 motorcycle, 261.4ms
Speed: 38.5ms preprocess, 261.4ms inference, 31.1ms postprocess per image at shape (1, 3, 320, 640)
0    332.312683
1    134.967468
2    370.122406
3    167.948288
4      0.767411
5      2.000000
Name: 0, dtype: float64
0    521.993164
1    229.807190
2    594.152405
3    306.001801
4      0.686606
5      2.000000
Name: 1, dtype: float64
0    478.792572
1     90.859871
2    503.113983
3    108.493111
4      0.604589
5      2.000000
Name: 2, dtype: float64
0    516.215027
1     92.567833
2    537.117920
3    108.750809
4      0.539120
5      2.000000
Name: 3, dtype: float64
0    209.631638
1    181.529297
2    225.575470
3    216.381927
4      0.463172
5      0.000000
Name: 4, dtype: float64
0    858.878906
1    252.336151
2    967.623169
3    316.431213
4      0.453826
5      2.000000
Name: 5, dtype: float64
0     991.951965
1     245.712250
2    1019.716125
3     306.706360
4       0.409927
5       2.000000
Name: 6, dtype: float64
0